# Data Gathering and EDA - Monthly Timeframes

This notebook deals exclusively with the series that have monthly time scales, which is our target time scale.

In [1]:
# Standard Library Modules
import json
import os
from pathlib import Path

# Pip Modules
import pandas as pd
from requests import HTTPError
import matplotlib.pyplot as plt

# Custom Modules
from src.utilities import new_logger, fetch_with_cache


# Setting Pandas DataFrame options
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

In [2]:
# Start the logging object
logger = new_logger("eda.monthly_series", 'logs/eda')

In [3]:
# Storing a JSON configuration file at the root of the project for EDA
# will use the config.yaml with Hydra during the actual ML pipeline
config_path = Path('fred_api.conf')
abs_config_path = config_path.resolve()

# API Configuration
if os.path.exists(abs_config_path):
    logger.debug(f"Discovered {abs_config_path}, attempting to read...")
    with open(abs_config_path, 'r') as json_fp:
        logger.debug(f"Opened {abs_config_path}, attempting to load.")
        config = json.load(json_fp)
        logger.debug(f"Loaded {abs_config_path}, checking attributes...")
    if not(isinstance(config['api_uri'], str) and len(config['api_uri']) > 0):
        logger.error(f"The JSON config is missing the attribute 'api_uri', please make sure it exists.")
    elif not(isinstance(config['api_key'], str) and len(config['api_key']) > 0):
        logger.error(f"The JSON config is missing the attribute 'api_key', please make sure it exists.")
    else:
        logger.info(f"All attributes found, you may continue.")
else:
    logger.error(f"Could not find {abs_config_path}, make sure it exists before continuing.")

## Monthly Series

The monthly series are going to be the easiest to deal with. For each series, the goal is to bring down the original response and save it as a Feather file under its series name.

If the Feather file is older than 30 days, the data will be refreshed. Otherwise, the feather file on disk will be used. This is to reduce the number of times the API is called.

In [4]:
monthly_series = [
    'HSN1FNSA',
    'PERMIT1NSA',
    'HOUST1FNSA',
    'UNDCON1UNSA',
    'COMPU1UNSA',
    'ACTLISCOUUS',
    'NEWLISCOUUS',
    'MEDDAYONMARUS',
    'MNMFS',
    'EXSFHSUSM495N',
    'HOSINVUSM495N',
    'MSACSRNSA',
    'PRRESCON',
    'WPU80',
    'PPIACO',
    'WPU101',
    'WPU102',
    'WPU081',
    'WPU139902094',
    'FMNHSHPSIUS',
    'FIXHAI',
    'UNRATE',
    'ADPMINDCONNERNSA',
    'ADPMNUSNERNSA',
    'CSUSHPINSA',
    'UMCSENT',
    'CUUR0000SEHA'
]

The approach here is relatively straightforward:

1. Make an API GET request for the series in question.
2. Save the resulting JSON response in CSV format under data/orig/
3. Parse the JSON response object:
    1. Index can be found in `resp['observations'][i]['date']`, where i is the index of the observation
    2. Value can be found in `resp['observations'][i]['value']`, where i is the index of the observation
4. Create a DataFrame, convert the `date` column to a `np.datetime64[ns]` and make it the index.

In [5]:
# proof of concept using a single series
series_id = 'HSN1FNSA'
request_uri = f"{config['api_uri']}?series_id={series_id}&api_key={config['api_key']}&file_type=json"

series_df = fetch_with_cache(series_id=series_id, request_uri=request_uri, dest="data/orig")

In [6]:
series_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 752 entries, 0 to 751
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   date      752 non-null    datetime64[ns]
 1   HSN1FNSA  752 non-null    float64       
dtypes: datetime64[ns](1), float64(1)
memory usage: 11.9 KB


In [7]:
series_df.head(15)

,date,HSN1FNSA
0,1963-01-01,42.0
1,1963-02-01,35.0
2,1963-03-01,44.0
3,1963-04-01,52.0
4,1963-05-01,58.0
5,1963-06-01,48.0
6,1963-07-01,62.0
7,1963-08-01,56.0
8,1963-09-01,49.0
9,1963-10-01,44.0


Now that the single-series POC is out of the way, let's dive right in. I will create a list of dataframes and then parse that list so that we end up with one large dataframe comprised of all monthly features.

The `date` columns will be used to outer join all of the series together. Then, any na rows will be dropped and we'll see what we're left with.

In [8]:
# fetching all of the data down and caching it locally
data_frames = []
for series in monthly_series:
    request_uri = f"{config['api_uri']}?series_id={series}&api_key={config['api_key']}&file_type=json"
    logger.info(f"Starting fetch process for {series}...")
    try:
        data_frames.append(fetch_with_cache(series_id=series, request_uri=request_uri, dest="data/orig"))
        logger.debug(f"Setting the {series} DataFrame's index to `date`...")
        data_frames[-1] = data_frames[-1].set_index('date', drop=True).sort_index()
        logger.debug(data_frames[-1].head(2))
    except HTTPError as err:
        logger.error(err)
    logger.info(f"Fetch process for {series} ({data_frames[-1].shape}) is complete.")

In [9]:
monthly_df = pd.concat(data_frames, axis=1, join='outer', verify_integrity=True)

In [10]:
monthly_df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1354 entries, 1913-01-01 to 2025-10-01
Freq: MS
Data columns (total 27 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   HSN1FNSA          752 non-null    float64
 1   PERMIT1NSA        800 non-null    float64
 2   HOUST1FNSA        800 non-null    float64
 3   UNDCON1UNSA       669 non-null    float64
 4   COMPU1UNSA        692 non-null    float64
 5   ACTLISCOUUS       112 non-null    float64
 6   NEWLISCOUUS       112 non-null    float64
 7   MEDDAYONMARUS     112 non-null    float64
 8   MNMFS             608 non-null    float64
 9   EXSFHSUSM495N     13 non-null     float64
 10  HOSINVUSM495N     13 non-null     float64
 11  MSACSRNSA         752 non-null    float64
 12  PRRESCON          392 non-null    float64
 13  WPU80             195 non-null    float64
 14  PPIACO            1352 non-null   float64
 15  WPU101            1196 non-null   float64
 16  WPU102         

In [11]:
monthly_df.describe()

,HSN1FNSA,PERMIT1NSA,HOUST1FNSA,UNDCON1UNSA,COMPU1UNSA,ACTLISCOUUS,NEWLISCOUUS,MEDDAYONMARUS,MNMFS,EXSFHSUSM495N,HOSINVUSM495N,MSACSRNSA,PRRESCON,WPU80,PPIACO,WPU101,WPU102,WPU081,WPU139902094,FMNHSHPSIUS,FIXHAI,UNRATE,ADPMINDCONNERNSA,ADPMNUSNERNSA,CSUSHPINSA,UMCSENT,CUUR0000SEHA
count,752.000000,800.000000,800.000000,669.000000,692.000000,1.120000e+02,112.000000,112.000000,608.000000,13.000000,1.300000e+01,752.000000,392.000000,195.000000,1352.000000,1196.000000,1196.000000,1196.000000,129.000000,175.00000,13.000000,932.000000,1.900000e+02,1.900000e+02,464.000000,665.000000,1069.000000
mean,54.695479,74.559875,84.508250,563.311958,83.548410,9.096872e+05,421838.892857,57.125000,4.957401,306230.769231,1.392308e+06,6.191622,38351.423469,122.773544,79.141930,92.644375,100.473963,97.921573,133.076333,76.56800,100.000000,5.670386,6.748211e+06,1.210996e+08,150.879578,84.898947,129.728935
std,18.315596,26.090490,27.833287,157.006120,24.641903,2.983720e+05,82726.680860,12.728718,2.003155,43380.398504,1.472047e+05,1.916582,18515.887182,23.595153,71.397638,90.969971,93.284567,86.243915,20.721828,9.24815,3.263944,1.707418,9.777278e+05,8.503040e+06,71.486786,13.217724,110.390731
min,20.000000,22.100000,22.700000,221.600000,26.000000,3.465140e+05,215940.000000,30.000000,1.500000,218000.000000,1.140000e+06,3.000000,12141.000000,97.600000,10.300000,8.500000,7.600000,4.500000,100.000000,56.70000,93.800000,2.500000,5.126000e+06,1.038990e+08,63.733000,50.000000,21.000000
25%,43.000000,56.300000,63.475000,470.500000,68.175000,6.687245e+05,372829.000000,50.000000,3.700000,287000.000000,1.330000e+06,4.800000,23479.500000,104.250000,18.100000,20.350000,28.100000,27.900000,113.900000,69.65000,98.800000,4.300000,5.824250e+06,1.147122e+08,83.520250,74.500000,37.900000
50%,53.000000,71.550000,83.250000,559.600000,83.200000,9.283750e+05,426440.000000,57.000000,4.600000,314000.000000,1.370000e+06,5.900000,33469.500000,113.900000,35.400000,60.250000,69.850000,70.800000,134.100000,76.50000,100.900000,5.500000,6.931500e+06,1.211930e+08,144.018000,88.600000,85.800000
75%,64.000000,90.800000,102.525000,659.600000,99.400000,1.160874e+06,486274.000000,65.000000,5.700000,341000.000000,1.540000e+06,7.200000,49010.000000,131.423000,125.700000,125.350000,136.925000,172.200000,149.651000,83.35000,102.400000,6.700000,7.440500e+06,1.272350e+08,183.633500,94.700000,205.100000
max,127.000000,166.200000,170.400000,985.000000,162.800000,1.463025e+06,584354.000000,88.000000,14.000000,356000.000000,1.550000e+06,14.400000,88297.000000,168.824000,280.251000,433.525000,403.846000,462.000000,170.304000,93.80000,104.900000,14.800000,8.479000e+06,1.359950e+08,331.685000,112.000000,438.212000


In [12]:
monthly_df[monthly_df.notna()]

,HSN1FNSA,PERMIT1NSA,HOUST1FNSA,UNDCON1UNSA,COMPU1UNSA,ACTLISCOUUS,NEWLISCOUUS,MEDDAYONMARUS,MNMFS,EXSFHSUSM495N,HOSINVUSM495N,MSACSRNSA,PRRESCON,WPU80,PPIACO,WPU101,WPU102,WPU081,WPU139902094,FMNHSHPSIUS,FIXHAI,UNRATE,ADPMINDCONNERNSA,ADPMNUSNERNSA,CSUSHPINSA,UMCSENT,CUUR0000SEHA
date,,,,,,,,,,,,,,,,,,,,,,,,,,,
1913-01-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12.100,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1913-02-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1913-03-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1913-04-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1913-05-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11.900,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1913-06-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11.900,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1913-07-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1913-08-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1913-09-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12.200,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Weekly and Daily Series

We can use the FRED frequency aggregation feature to convert higher frequency data series into lower frequency data series.

This is done with the `frequency` and `aggregation_method` parameters in the API request.

In [13]:
hf_series = [
    'MORTGAGE30US',
    'MORTGAGE15US',
    'OBMMIVA30YF',
    'OBMMIJUMBO30YF',
    'OBMMIFHA30YF',
    'OBMMIC30YF',
    'OBMMIUSDA30YF',
    'OBMMIC30YFNA',
    'OBMMIC15YF'
]

In [14]:
# fetching all of the higher frequency data down and caching it locally
hf_data_frames = []
for series in hf_series:
    request_uri = f"{config['api_uri']}?series_id={series}&api_key={config['api_key']}&file_type=json&frequency=m&aggregation_method=eop"
    logger.info(f"Starting fetch process for {series}...")
    try:
        hf_data_frames.append(fetch_with_cache(series_id=series, request_uri=request_uri, dest="data/orig"))
        logger.debug(f"Setting the {series} DataFrame's index to `date`...")
        hf_data_frames[-1] = hf_data_frames[-1].set_index('date', drop=True).sort_index()
        logger.debug(hf_data_frames[-1].head(2))
    except HTTPError as err:
        logger.error(err)
    logger.info(f"Fetch process for {series} ({hf_data_frames[-1].shape}) is complete.")

In [15]:
monthly_hf_df = pd.concat(hf_data_frames, axis=1, join='outer', verify_integrity=True)

In [16]:
monthly_hf_df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 656 entries, 1971-04-01 to 2025-11-01
Freq: MS
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   MORTGAGE30US    655 non-null    float64
 1   MORTGAGE15US    410 non-null    float64
 2   OBMMIVA30YF     106 non-null    float64
 3   OBMMIJUMBO30YF  106 non-null    float64
 4   OBMMIFHA30YF    106 non-null    float64
 5   OBMMIC30YF      106 non-null    float64
 6   OBMMIUSDA30YF   106 non-null    float64
 7   OBMMIC30YFNA    106 non-null    float64
 8   OBMMIC15YF      106 non-null    float64
dtypes: float64(9)
memory usage: 51.2 KB


In [17]:
monthly_hf_df.index.values

array(['1971-04-01T00:00:00.000000000', '1971-05-01T00:00:00.000000000',
       '1971-06-01T00:00:00.000000000', '1971-07-01T00:00:00.000000000',
       '1971-08-01T00:00:00.000000000', '1971-09-01T00:00:00.000000000',
       '1971-10-01T00:00:00.000000000', '1971-11-01T00:00:00.000000000',
       '1971-12-01T00:00:00.000000000', '1972-01-01T00:00:00.000000000',
       '1972-02-01T00:00:00.000000000', '1972-03-01T00:00:00.000000000',
       '1972-04-01T00:00:00.000000000', '1972-05-01T00:00:00.000000000',
       '1972-06-01T00:00:00.000000000', '1972-07-01T00:00:00.000000000',
       '1972-08-01T00:00:00.000000000', '1972-09-01T00:00:00.000000000',
       '1972-10-01T00:00:00.000000000', '1972-11-01T00:00:00.000000000',
       '1972-12-01T00:00:00.000000000', '1973-01-01T00:00:00.000000000',
       '1973-02-01T00:00:00.000000000', '1973-03-01T00:00:00.000000000',
       '1973-04-01T00:00:00.000000000', '1973-05-01T00:00:00.000000000',
       '1973-06-01T00:00:00.000000000', '1973-07-01

In [18]:
mseries_df = pd.concat([monthly_df, monthly_hf_df], axis=1, join="outer", verify_integrity=True)

In [19]:
mseries_df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1355 entries, 1913-01-01 to 2025-11-01
Freq: MS
Data columns (total 36 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   HSN1FNSA          752 non-null    float64
 1   PERMIT1NSA        800 non-null    float64
 2   HOUST1FNSA        800 non-null    float64
 3   UNDCON1UNSA       669 non-null    float64
 4   COMPU1UNSA        692 non-null    float64
 5   ACTLISCOUUS       112 non-null    float64
 6   NEWLISCOUUS       112 non-null    float64
 7   MEDDAYONMARUS     112 non-null    float64
 8   MNMFS             608 non-null    float64
 9   EXSFHSUSM495N     13 non-null     float64
 10  HOSINVUSM495N     13 non-null     float64
 11  MSACSRNSA         752 non-null    float64
 12  PRRESCON          392 non-null    float64
 13  WPU80             195 non-null    float64
 14  PPIACO            1352 non-null   float64
 15  WPU101            1196 non-null   float64
 16  WPU102         

In [20]:
mseries_df.tail(24)

,HSN1FNSA,PERMIT1NSA,HOUST1FNSA,UNDCON1UNSA,COMPU1UNSA,ACTLISCOUUS,NEWLISCOUUS,MEDDAYONMARUS,MNMFS,EXSFHSUSM495N,HOSINVUSM495N,MSACSRNSA,PRRESCON,WPU80,PPIACO,WPU101,WPU102,WPU081,WPU139902094,FMNHSHPSIUS,FIXHAI,UNRATE,ADPMINDCONNERNSA,ADPMNUSNERNSA,CSUSHPINSA,UMCSENT,CUUR0000SEHA,MORTGAGE30US,MORTGAGE15US,OBMMIVA30YF,OBMMIJUMBO30YF,OBMMIFHA30YF,OBMMIC30YF,OBMMIUSDA30YF,OBMMIC30YFNA,OBMMIC15YF
date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2023-12-01,49.0,66.3,72.8,658.1,100.6,714214.0,235584.0,61.0,2.5,NaN,NaN,9.2,63991.0,163.514,249.866,325.503,323.022,244.746,158.904,67.2,NaN,3.8,7923000.0,132439000.0,310.940,69.7,410.606,6.61,5.93,6.174,6.514,6.485,6.564,6.540,6.408,5.993
2024-01-01,58.0,77.2,70.3,664.6,61.8,665603.0,295190.0,69.0,2.7,NaN,NaN,7.9,62796.0,163.998,251.306,338.362,321.658,246.536,159.199,70.7,NaN,3.7,7877000.0,131172000.0,310.809,79.0,412.019,6.69,5.96,6.091,6.973,6.370,6.528,6.360,6.377,5.861
2024-02-01,58.0,80.5,82.2,664.2,81.6,664745.0,339380.0,61.0,2.5,NaN,NaN,7.8,63898.0,163.963,254.926,344.739,322.158,248.576,158.408,72.8,NaN,3.9,7867000.0,130546000.0,312.735,76.9,413.695,6.94,6.26,6.500,7.347,6.655,6.885,6.766,6.752,6.344
2024-03-01,65.0,86.0,88.8,673.4,78.9,694844.0,395536.0,50.0,2.8,NaN,NaN,7.0,73435.0,164.201,255.095,321.137,327.703,251.805,157.704,71.9,NaN,3.9,7890000.0,130578000.0,316.919,79.4,415.219,6.79,6.11,6.312,6.931,6.455,6.731,6.450,6.610,6.004
2024-04-01,65.0,94.3,96.1,673.0,91.0,734324.0,432032.0,47.0,2.3,NaN,NaN,7.0,79721.0,164.816,256.978,314.128,336.562,254.976,161.653,71.9,NaN,3.9,7942000.0,131024000.0,320.848,77.2,416.386,7.17,6.44,6.798,7.396,6.909,7.242,6.825,7.120,6.632
2024-05-01,61.0,94.4,91.6,676.6,86.2,787726.0,433906.0,45.0,2.4,NaN,NaN,7.6,85582.0,164.841,255.313,314.168,351.200,250.461,162.584,69.4,NaN,4.0,8048000.0,131666000.0,323.795,69.1,417.772,7.03,6.36,6.582,7.156,6.749,7.020,6.801,6.895,6.346
2024-06-01,58.0,83.8,93.3,671.3,93.1,839960.0,426092.0,48.0,2.1,NaN,NaN,8.1,87280.0,165.347,255.914,308.611,351.993,250.044,161.738,72.6,NaN,4.1,8168000.0,133217000.0,325.348,68.2,418.820,6.86,6.16,6.462,7.182,6.665,6.938,6.711,6.749,6.379
2024-07-01,61.0,85.7,80.9,658.6,91.3,883905.0,405404.0,51.0,2.2,NaN,NaN,7.7,87211.0,166.518,257.321,301.282,351.282,247.533,162.229,71.5,93.8,4.2,8274000.0,134401000.0,325.670,66.4,420.577,6.78,6.07,6.146,6.979,6.532,6.674,6.466,6.513,5.926
2024-08-01,56.0,85.8,89.5,657.0,89.3,908944.0,383552.0,53.0,2.2,NaN,NaN,8.5,87396.0,166.513,255.463,295.464,342.036,251.601,161.640,72.1,99.1,4.2,8309000.0,134503000.0,325.113,67.9,422.223,6.35,5.51,5.786,6.639,6.130,6.368,6.212,6.192,5.677
